# 01 - Instalando K-Modes

In [11]:
!pip install kmodes

# 02 - Importando bibliotecas

In [12]:
import pandas as pd
import numpy as np
from kmodes.kmodes import KModes
from sklearn.metrics.cluster import rand_score
from sklearn.metrics.cluster import adjusted_rand_score

import collections
import statistics

pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

# 03 - Métodos e funções (validação e similaridade)

In [13]:
def overlap(list1, list2):
  if len(srag.iloc[1]) == len(srag.iloc[1]):
    weight = 1/len(srag.iloc[1])
    return (float(sum(x != y for x, y in zip(list1, list2)))*weight)

In [14]:
def constroi_centroides(dataframe, grupos, features):
  # anexa os grupos obtidos pelo KModes ao dataframe
  dataframe = dataframe.assign(grupos=grupos)
  # cria vetor de centroides que abrigará os centroides
  # os centroides serão calculados a partir da moda de um subconjunto de dados.
  centroides_ = []
  grupos_i = dataframe['grupos'].unique()
  # para cada grupo, calcular a moda e criar um atributo "grupo"
  for grupo in grupos_i:
    datatemp = dataframe.loc[dataframe['grupos'] == grupo][features].mode()
    # datatemp['grupos'] = grupo
    datatemp = datatemp.assign(grupos=grupo)
    # adicionar esta linha com modas como um centroide na lista
    centroides_.append(datatemp)
  # gerar um dataframe com todo os centroides
  centroides = pd.concat(centroides_)
  # gerar indices com o numero do grupo e excluindo o atributo grupo.
  centroides = centroides.set_index([grupos_i])
  centroides = centroides.drop(columns=['grupos'])
  # retornando resultado
  return centroides

In [15]:
def calcula_distancia_ahmad(dict1_registro, dict2_grupo):
  # Estabelecendo pesos para cada atributo
  weight = 1.0/len(dict1_registro)
  # iniciando vetor com distancias para somatorio
  distancias = []
  # criando as chaves para consulta em dicionários de distancias
  for col in dict1_registro.keys():
    valor_registro = "'{}'".format(dict1_registro[col])
    valor_grupo = "'{}'".format(dict2_grupo[col])
    # caso valor seja igual, distancia sera 0.0
    if valor_registro == valor_grupo:
      distancias.append(0.0)
    else:
      # em caso de valores diferentes, encontrar a chave no dicionário
      key = "[{},{}]".format(valor_registro, valor_grupo)
      # fazendo consulta no dicionario de distancias
      distancia = mapa_de_distancias[col]['distances'][key]
      # incluindo distancia no vetor
      distancias.append(distancia*weight)
  # finalizando
  return sum(distancias)

In [16]:
def calcula_a_e_b(registro_agrupado, dado_agrupado, centroides, features):
  # inicia vetor de distancias
  distancias = []
  # verifica grupo ao qual o registro pertence
  grupo_registro = registro_agrupado['grupos']
  # considerando as features, transforma o registro em dicionários
  registro_ = registro_agrupado[features].to_dict()
  # verifica id do grupo do registro e transforma ele em dict
  grupo = centroides.iloc[int(grupo_registro)].to_dict()
  # inicia vetor de distancias entre registro e
  # centroides de outros grupos
  b = []
  # verificando se registro e centroides tem os mesmos atributos

  # precorrendo os centroides
  for id_centroide in centroides.index:
    if collections.Counter(registro_.keys()) == collections.Counter(grupo.keys()):
      # se o centroide for o mesmo do registro, calcular "a"
      # a = distancia entre o registro e o centroide que o representa
      if id_centroide == grupo_registro:
        a = calcula_distancia_ahmad(registro_, grupo)
      # senao, calcular "b"
      # b = distancia entre registro e demais centroides
      else:
        # verificar os valores do centroide de outro grupo e transformando em dict
        grupo_ = centroides.iloc[int(id_centroide)].to_dict()
        # incrementa vetor de distancias "b"
        b.append(calcula_distancia_ahmad(registro_, grupo_))
    else:
      print("Dados com dimensões diferentes:\n{}\{}".format(registro_, grupo))
      return None
  # retornar valor
  return a, min(b), grupo_registro

In [17]:
def calcula_sw(dado_agrupado, centroides, features):
  # define dicionario para coletar a silhueta em cada registro
  dicionario_sw = {}
  sw_ = []
  for id_centroide in centroides.index:
    dicionario_sw[id_centroide] = {}
  # percorrendo os dados
  for id, registro_agrupado in dado_agrupado.iterrows():
    # calculando a metricas a (distancia com centroide do grupo) e
    # b (distancia minima com centroide de outros grupos)
    a, b, grupo = calcula_a_e_b(registro_agrupado, dado_agrupado, centroides, features)
    # calculando silhueta
    sw = (b - a)/max(a, b)
    # incluindo sw no dicionario
    dicionario_sw[grupo][id] = sw
    sw_.append(sw)
  # calculando silhueta media
  S = statistics.mean(sw_)
  return dicionario_sw, S

In [18]:
# Complexidade quadrática impede execução.
# Numa base como esta, com 189893 registros,
# a estrutura (n de linhas e colunas, Nxp) resultante
# seria de 189893 x 189893, 36.059.351.449 pontos de dados.
# def matriz_dissimilaridade(data):
#   df_cartesiano = {}
#   for id_i in data.index:
#     df_cartesiano[id_i] = []
#     for id_j in data.index:
#       record_i = data.iloc[id_i]
#       record_j = data.iloc[id_j]
#       df_cartesiano[id_i].append(overlap(record_i, record_j))
#   return df_cartesiano

# matriz = matriz_dissimilaridade(srag_)

# 04 - Lendo base de dados

In [19]:
srag = pd.read_csv('/content/drive/MyDrive/srag_dataset_2013_a_2023-attr_set_3.csv')
# srag = srag.sample(n = 1000, random_state=2023)
srag.head()

,PERD_OLFT,PERD_PALA,FEBRE,DISPINEIA,SATURACAO,FADIGA,SUPORT_VEN,RAIOX_RES,TOSSE,TOMO_RES,UTI,GARGANTA,CS_SEXO,DIST_PRI_INTERN,TEMPO_UTI,POSSUI_MORBIDADE,FAIXA_ETARIA,FAIXA_ETARIA_2,IMAGEM_RES,VACINAS,ANO,POSSIVEL_VARIANTE,EVOLUCAO,CRITERIO,CLASSI_FIN,id
0,9.0,9.0,1.0,9.0,2.0,9.0,3.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,9.0,2.0,11.0,3.0,2.0,0.0,2020.0,1.0,9.0,4.0,5.0,206158430208
1,9.0,9.0,9.0,9.0,9.0,9.0,3.0,5.0,9.0,9.0,1.0,9.0,2.0,1.0,9.0,2.0,2.0,3.0,3.0,0.0,2020.0,1.0,9.0,4.0,5.0,206158430209
2,9.0,9.0,1.0,9.0,1.0,9.0,2.0,9.0,1.0,9.0,1.0,9.0,2.0,1.0,9.0,2.0,11.0,3.0,0.0,0.0,2020.0,1.0,2.0,3.0,5.0,206158430210
3,9.0,9.0,1.0,9.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,8.0,2.0,2.0,0.0,2020.0,1.0,1.0,4.0,5.0,206158430211
4,9.0,9.0,1.0,9.0,2.0,9.0,3.0,2.0,1.0,9.0,2.0,1.0,2.0,1.0,9.0,0.0,6.0,2.0,0.0,0.0,2020.0,1.0,1.0,3.0,5.0,206158430212


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 05 - Selecionando variáveis

In [20]:
srag.columns

Index(['PERD_OLFT', 'PERD_PALA', 'FEBRE', 'DISPINEIA', 'SATURACAO', 'FADIGA',
       'SUPORT_VEN', 'RAIOX_RES', 'TOSSE', 'TOMO_RES', 'UTI', 'GARGANTA',
       'CS_SEXO', 'DIST_PRI_INTERN', 'TEMPO_UTI', 'POSSUI_MORBIDADE',
       'FAIXA_ETARIA', 'FAIXA_ETARIA_2', 'IMAGEM_RES', 'VACINAS', 'ANO',
       'POSSIVEL_VARIANTE', 'EVOLUCAO', 'CRITERIO', 'CLASSI_FIN', 'id'],
      dtype='object')

In [21]:
features = ['PERD_OLFT','FEBRE','DISPINEIA', 'SATURACAO' , 'FADIGA', 'PERD_PALA', 'SUPORT_VEN', 'RAIOX_RES', 'GARGANTA', 'TOMO_RES', 'FAIXA_ETARIA_2',
            'IMAGEM_RES', 'ANO','POSSUI_MORBIDADE', 'EVOLUCAO', 'CLASSI_FIN', 'UTI', 'TEMPO_UTI']

In [22]:
srag_ = srag[features]
srag_.head(3)

,PERD_OLFT,FEBRE,DISPINEIA,SATURACAO,FADIGA,PERD_PALA,SUPORT_VEN,RAIOX_RES,GARGANTA,TOMO_RES,FAIXA_ETARIA_2,IMAGEM_RES,ANO,POSSUI_MORBIDADE,EVOLUCAO,CLASSI_FIN,UTI,TEMPO_UTI
0,9.0,1.0,9.0,2.0,9.0,9.0,3.0,2.0,2.0,1.0,3.0,2.0,2020.0,2.0,9.0,5.0,1.0,9.0
1,9.0,9.0,9.0,9.0,9.0,9.0,3.0,5.0,9.0,9.0,3.0,3.0,2020.0,2.0,9.0,5.0,1.0,9.0
2,9.0,1.0,9.0,1.0,9.0,9.0,2.0,9.0,9.0,9.0,3.0,0.0,2020.0,2.0,2.0,5.0,1.0,9.0


# 06 - Executando modelo

In [23]:
# KModes?

In [24]:
km = KModes(n_clusters=3, init='Huang', n_init=1, max_iter=5, n_jobs=-1, verbose=1)

In [ ]:
clusters = km.fit_predict(srag_)

In [ ]:
collections.Counter(clusters)

# 07 - Validando modelo (critério externo)

## Critério Externo (Rand Index)

In [ ]:
rand_score(srag['POSSIVEL_VARIANTE'], clusters)

## Critério relativo (Silhueta)

In [ ]:
import json
f = open('/content/drive/MyDrive/srag_2013_a_2023_dataset-attr_set_3-ahmad-distance-matrix.json.csv')
mapa_de_distancias = json.load(f)

In [ ]:
# mapa_de_distancias

In [ ]:
srag_predicted = srag[features]
# clusters_ = pd.Series(clusters)
srag_predicted = srag_predicted.assign(grupos=clusters)
# srag_predicted.insert(loc=, column='grupos', value=clusters_)
# srag_predicted['grupos'] = clusters
srag_predicted.head(2)

In [ ]:
centroides = constroi_centroides(srag[features], clusters, features)
centroides

In [ ]:
sw = calcula_sw(srag_predicted, centroides, features)
sw_pd = pd.concat({k: pd.DataFrame.from_dict(v, 'index') for k, v in sw[0].items()}, axis=0).reset_index().rename(columns={'level_0': 'grupo', 'level_1': 'id_registro', 0: 'silhueta'})

print("Silhueta media: {}".format(sw[1]))

In [ ]:
sw_pd

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

figure, axis = plt.subplots(3)

figure.suptitle('Silhueta Kmodes - SRAG', fontsize=8)
figure.supxlabel('sw', fontsize=8)
figure.supylabel('Registros agrupados', fontsize=8)

plt.rcParams["figure.figsize"] = [4.0, 7.50]
plt.rcParams["figure.autolayout"] = True

# cluster 0
# create dataset
data1 = sw_pd.loc[sw_pd['grupo'] == 0].sort_values(by=['silhueta'])
height1 = data1['silhueta']
bars1 = data1['id_registro']
y_pos1 = np.arange(len(bars1))
# Create horizontal bars
axis[0].set_title('cluster 0', fontsize=4)
axis[0].axis(xmin=-1,xmax=1)
axis[0].barh(y_pos1, height1)
axis[0].plot()


# cluster 1
# create dataset
data2 = sw_pd.loc[sw_pd['grupo'] == 1].sort_values(by=['silhueta'])
height2 = data2['silhueta']
bars2 = data2['id_registro']
y_pos2 = np.arange(len(bars2))
# Create horizontal bars
axis[1].set_title('cluster 1', fontsize=4)
axis[1].axis(xmin=-1,xmax=1)
axis[1].barh(y_pos2, height2)
axis[1].plot()

# cluster 2
# create dataset
data3 = sw_pd.loc[sw_pd['grupo'] == 2].sort_values(by=['silhueta'])
height3 = data3['silhueta']
bars3 = data3['id_registro']
y_pos3 = np.arange(len(bars3))
# Create horizontal bars
axis[2].set_title('cluster 2', fontsize=4)
axis[2].axis(xmin=-1,xmax=1)
axis[2].barh(y_pos3, height3)
axis[2].plot()

# Show graphic
plt.show()

In [ ]:
# Como podemos melhorar o resultado do rand_score?
# Qual seria a aplicação da silhueta?

# Exemplo de Silhueta em K-means

In [ ]:
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
#
# Load IRIS dataset
#
iris = datasets.load_iris()
X = iris.data
y = iris.target
#
# Instantiate the KMeans models
#
km = KMeans(n_clusters=3, random_state=42)
#
# Fit the KMeans model
#
km.fit_predict(X)
#
# Calculate Silhoutte Score
#
score = silhouette_score(X, km.labels_, metric='euclidean')
#
# Print the score
#
print('Silhouetter Score: %.3f' % score)

In [ ]:
# from yellowbrick.cluster import SilhouetteVisualizer

# fig, ax = plt.subplots(2, 2, figsize=(15,8))
# for i in [2, 3, 4, 5]:
#     '''
#     Create KMeans instance for different number of clusters
#     '''
#     km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=42)
#     q, mod = divmod(i, 2)
#     '''
#     Create SilhouetteVisualizer instance with KMeans instance
#     Fit the visualizer
#     '''
#     visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[q-1][mod])
#     visualizer.fit(X)

In [ ]:
km

In [ ]:
SilhouetteVisualizer?